In [1]:
import requests

from utils.find_miner_id import ReadTokens

## TO DO

Miners will have various NFT's in wallet and we search for the token name:
if we find it, we grab the token-ID, we find out who minted this token, if the minter is the person who currently has the NFT we can assume this person is the miner setting their parameters.

In [2]:

url = 'https://api.ergo.aap.cornell.edu/api/v1/boxes/byAddress'
# url = 'https://api.ergoplatform.com/api/v1/boxes/byTokenId/'
token_id = '5104822ac703509fb6790961d8d2b4afa4c75b9514abfd48ca8bf83639d30574'
wallet = '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk'
# wallet = '9hAcdWpFAv7biCSeUcCvXWYRfEepm1ubdsfg5PC48k9S7ymiU3W'

find_tokens = ReadTokens(url)


In [3]:
tokens = find_tokens.find_token_name_in_wallet(wallet, 'Sigmanaut Mining Pool Miner ID - Season 0')
tokens

[{'tokenId': 'a24b57a4b9a8122a52c5d92a6250b7ed5df134e660f3e3946770858a3fc46eb7',
  'amount': 1,
  'decimals': 0,
  'name': 'Sigmanaut Mining Pool Miner ID - Season 0'},
 {'tokenId': '5104822ac703509fb6790961d8d2b4afa4c75b9514abfd48ca8bf83639d30574',
  'amount': 1,
  'decimals': 0,
  'name': 'Sigmanaut Mining Pool Miner ID - Season 0'},
 {'tokenId': '5b8a5d9a0335ac3785c3edb8356acfca4e172b738aaa9feadfafa6fdc9989634',
  'amount': 1,
  'decimals': 0,
  'name': 'Sigmanaut Mining Pool Miner ID - Season 0'}]

In [4]:
possible_tokens = find_tokens.find_token_name_in_wallet(wallet, 'Sigmanaut Mining Pool Miner ID - Season 0')

my_tokens = []
miner_id = None
for token in possible_tokens:
    
    id = token['tokenId']
    base_url = 'https://api.ergoplatform.com/api/v1/boxes/byTokenId'
    url = '{}/{}'.format(base_url, id)
    data = find_tokens.get_api_data(url)
    oldest_height = 1e30
    # for a given token we want to find the earliest TX with it so we can see who minted it
    for item in data['items']:
        if oldest_height > item['creationHeight']:
            oldest_height = item['creationHeight']
            # ls = item['boxId'], item['creationHeight'], item['transactionId'], item['address']
            tx = item['transactionId']
    
    # tx = ls[2]
    wallet_minted_tokens = []
    url = 'https://api.ergoplatform.com/api/v1/transactions'
    tx_url = '{}/{}'.format(url, tx)
    data = find_tokens.get_api_data(tx_url)
    # print(data)
    box_id_first_input = data['inputs'][0]['boxId']

    outputs = data['outputs']
    # print(outputs, 'oi')
    oldest_height = 0
    for sample in outputs:
        print(id, tx)
        try:   
            assets = sample['assets'][0]
            if assets['tokenId'] == box_id_first_input:
                minting_address = sample['address']
                if wallet == minting_address:
                    if sample['creationHeight'] > oldest_height:
                        oldest_height = sample['creationHeight']
                        my_tokens.append(token)
                        miner_id = token
                    break
        except IndexError:
            pass
            # wallet_minted_tokens.append(
    
# minting_address, id

a24b57a4b9a8122a52c5d92a6250b7ed5df134e660f3e3946770858a3fc46eb7 83626102ed1249872190ffb7c5bf33375ed2c27d0bec2c93db5897b19a31abd0
5104822ac703509fb6790961d8d2b4afa4c75b9514abfd48ca8bf83639d30574 90117ce9db18e1ed37be9cb491d464af408825ce4aaef8329880d7b6adfc1d46
5104822ac703509fb6790961d8d2b4afa4c75b9514abfd48ca8bf83639d30574 90117ce9db18e1ed37be9cb491d464af408825ce4aaef8329880d7b6adfc1d46
5104822ac703509fb6790961d8d2b4afa4c75b9514abfd48ca8bf83639d30574 90117ce9db18e1ed37be9cb491d464af408825ce4aaef8329880d7b6adfc1d46
5b8a5d9a0335ac3785c3edb8356acfca4e172b738aaa9feadfafa6fdc9989634 d84f73c421ec46b54374596ddeb8b4a56f2874f5a96ae659593423fa9ec3cac7


In [5]:
miner_id

{'tokenId': '5b8a5d9a0335ac3785c3edb8356acfca4e172b738aaa9feadfafa6fdc9989634',
 'amount': 1,
 'decimals': 0,
 'name': 'Sigmanaut Mining Pool Miner ID - Season 0'}

In [6]:
find_tokens.get_token_description(miner_id['tokenId'])

{'address': '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk',
 'height': 1299322,
 'minimumPayout': 33,
 'tokens': [{'token': 'ERGO', 'value': 50},
  {'token': 'SigUSD', 'value': 10},
  {'token': 'RSN', 'value': 10},
  {'token': 'rsBTC', 'value': 10},
  {'token': 'Gluon', 'value': 10},
  {'token': 'COMET', 'value': 5},
  {'token': 'Bober', 'value': 5}],
 'season': 0,
 'type': 'Miner ID',
 'fan_club': 'QX'}

In [7]:
find_tokens.get_token_description('5104822ac703509fb6790961d8d2b4afa4c75b9514abfd48ca8bf83639d30574')

{'address': '9eg7v2nkypUZbdyvSKSD9kg8FNwrEdTrfC2xdXWXmEpDAFEtYEn',
 'height': 1290164,
 'minimumPayout': 20,
 'tokens': [{'token': 'ERGO', 'value': 50},
  {'token': 'rsBTC', 'value': 20},
  {'token': 'SigUSD', 'value': 10},
  {'token': 'RSN', 'value': 10},
  {'token': 'CYPX', 'value': 10},
  {'token': '', 'value': 0}],
 'season': 0,
 'type': 'Miner ID',
 'fan_club': 'QX'}

In [8]:
token = find_tokens.get_latest_miner_id(wallet)


In [9]:
miner_id = find_tokens.get_token_description(token['tokenId'])[0]
miner_id

KeyError: 0

In [ ]:
import json

miner_id = json.loads(miner_id)
miner_id